# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [5]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [6]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [7]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [8]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [9]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [10]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [11]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [12]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [13]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [14]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [15]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [16]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [17]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [21]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'NVDA',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [22]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (NVDA). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for NVDA to the Data Analyst in order to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for NVDA using statistical modeling and machine learning", "context": "The goal is to identify trends and predict market movements for NVDA"}


> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for NVDA to analyze trends and predict market movements.

Action: Read website content
Action Input: {"website_url": "https://finance.yahoo.com/quote/NVDA"} 

NVIDIA Corporation (NVDA) Stock Price,

Final Answer: NVIDIA Corporation (NVDA) Stock Price, News, Quote & History - Yahoo Finance News Today's news US Politics World Tech Reviews and deals Audio Computing Gaming Health Home Phones Science TVs Climate change Health Science 2024 election Originals The 360 Life Health COVID-19 Fall allergies Health news Mental health Relax Sexual health Studies The Unwind Parenting Family health So mini ways Style and beauty It Figures Unapologetically Horoscopes Shopping Buying guides Food Travel Autos Gift ideas Buying guides Entertainment Celebrity TV Movies Music How to Watch Interviews Videos Shopping Finance My Portfolio My Portfolio News Latest News Stock Market Originals Premium News Economics Housing Earnings Tech Crypto Biden Economy Markets Stocks: Most Actives Stocks: Gainers Stocks: Losers Calendars Trending Tickers Futures World Indices US Treasury Bonds Currencies Crypto Top ETFs Top Mutual Funds Highest Open Interest Highest Implied Volatility Stock Comparison Advanced Charts C

Thought: I need to delegate the task of continuously monitoring and analyzing market data for NVDA to the Data Analyst in order to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for NVDA using statistical modeling and machine learning", "context": "The goal is to identify trends and predict market movements for NVDA"}
 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Thought: I need to ask the Data Analyst how they plan to continuously monitor and analyze market data for NVDA to identify trends and predict market movements.

Action: Ask question to co-worker
Action Input: {"coworker": "Data Analyst", "question": "How do you plan to continuously monitor and analyze market data for NVDA to identify trends and predict market movements?", "context": "The goal is to gather insights and alerts about signi

Final Answer: 
NVIDIA Corporation provides graphics and compute and networking solutions in the United States, Taiwan, China, Hong Kong, and internationally. The Graphics segment offers GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure, and solutions for gaming platforms; Quadro/NVIDIA RTX GPUs for enterprise workstation graphics; virtual GPU or vGPU software for cloud-based visual and virtual computing; automotive platforms for infotainment systems; and Omniverse software for building and operating metaverse and 3D internet applications. The Compute & Networking segment comprises Data Center computing platforms and end-to-end networking platforms, including Quantum for InfiniBand and Spectrum for Ethernet; NVIDIA DRIVE automated-driving platform and automotive development agreements; Jetson robotics and other embedded platforms; NVIDIA AI Enterprise and other software; and DGX Cloud software and services. The company's products are used

Final Answer: Based on the insights gathered from the NVIDIA website content, it is clear that NVIDIA is a leader in artificial intelligence computing and offers a wide range of solutions across various industries. The company's focus on AI, digital twins, and generative AI is transforming industries and impacting society in profound ways. To develop and refine trading strategies for NVDA, we will need to closely monitor NVIDIA's advancements in AI, digital twins, and edge computing applications, as these areas are likely to influence NVDA's stock performance.

> Finished chain.
 

Based on the insights gathered from the NVIDIA website content, it is clear that NVIDIA is a leader in artificial intelligence computing and offers a wide range of solutions across various industries. The company's focus on AI, digital twins, and generative AI is transforming industries and impacting society in profound ways. To develop and refine trading strategies for NVDA, we will need to closely monitor 

Final Answer: Based on the website content of NVIDIA, it does not directly provide information on user trading preferences or risk tolerance for NVDA. Additional research or data from other sources would be needed to determine the user's trading preferences and risk tolerance for NVDA.

> Finished chain.
 

Based on the website content of NVIDIA, it does not directly provide information on user trading preferences or risk tolerance for NVDA. Additional research or data from other sources would be needed to determine the user's trading preferences and risk tolerance for NVDA.

Final Answer: Based on the insights gathered from the NVIDIA website content, it is clear that NVIDIA is a leader in artificial intelligence computing and offers a wide range of solutions across various industries. The company's focus on AI, digital twins, and generative AI is transforming industries and impacting society in profound ways. To develop and refine trading strategies for NVDA, we will need to closely 

Final Answer: 

Based on the insights gathered from the NVIDIA website content, it is evident that NVIDIA is a leader in artificial intelligence computing. They offer a wide range of solutions across various industries, focusing on AI, digital twins, and generative AI. To develop and refine trading strategies for NVDA, it is crucial to closely monitor NVIDIA's advancements in these areas as they are likely to influence NVDA's stock performance.

> Finished chain.
 

Based on the insights gathered from the NVIDIA website content, it is evident that NVIDIA is a leader in artificial intelligence computing. They offer a wide range of solutions across various industries, focusing on AI, digital twins, and generative AI. To develop and refine trading strategies for NVDA, it is crucial to closely monitor NVIDIA's advancements in these areas as they are likely to influence NVDA's stock performance.

Thought: 
I need to ensure that the team thoroughly analyzes the approved trading strategies fo

Final Answer: NVIDIA is a leader in artificial intelligence computing, offering solutions across various industries. Their focus on AI, digital twins, and generative AI is transforming industries and impacting society in profound ways. To develop and refine trading strategies for NVDA, we need to closely monitor NVIDIA's advancements in AI, digital twins, and edge computing applications.

> Finished chain.
 

NVIDIA is a leader in artificial intelligence computing, offering solutions across various industries. Their focus on AI, digital twins, and generative AI is transforming industries and impacting society in profound ways. To develop and refine trading strategies for NVDA, we need to closely monitor NVIDIA's advancements in AI, digital twins, and edge computing applications.

Final Answer: 

Detailed execution plans suggesting how and when to execute trades for NVDA should be developed based on the insights gathered from the NVIDIA website content. NVIDIA is a leader in artificial 

Final Answer: 
Based on the insights gathered from the NVIDIA website content, it is clear that NVIDIA is a world leader in artificial intelligence computing. Their focus on AI, digital twins, and generative AI is transforming industries and impacting society in profound ways. To evaluate the risks associated with proposed trading strategies for NVDA, it is crucial to closely monitor NVIDIA's advancements in these areas. By analyzing the approved trading strategies for NVDA and considering the current market conditions, detailed execution plans should be developed to guide how and when to execute trades for NVDA, taking into account the company's advancements in AI, digital twins, and edge computing applications.

> Finished chain.
 

Based on the insights gathered from the NVIDIA website content, it is clear that NVIDIA is a world leader in artificial intelligence computing. Their focus on AI, digital twins, and generative AI is transforming industries and impacting society in profoun

- Display the final result as Markdown.

In [20]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for AAPL Trading Strategies:

After evaluating the risks associated with the proposed trading strategies for AAPL, it is evident that market volatility, company-specific risks, and external factors play a significant role in determining the success of the trading plans. 

1. Aggressive Strategy:
- Market Volatility: The aggressive strategy heavily relies on short-term trading opportunities, making it susceptible to rapid price movements. Utilizing options and margin trading can amplify returns but also increase the risk of losses.
- Company-Specific Risks: Apple's strong financial performance provides a solid foundation, but any unexpected events or changes in consumer preferences can impact stock prices.
- Mitigation Strategies: Diversifying the portfolio, implementing tight stop-loss orders, and staying updated with market news can help mitigate risks associated with the aggressive strategy.

2. Moderate Strategy:
- External Factors: Integrating technical and fundamental analysis in the moderate strategy provides a balanced approach, but external factors like regulatory changes and market sentiment can introduce uncertainties.
- Mitigation Strategies: Regularly reviewing and adjusting the trading strategy based on market conditions, diversifying the portfolio, and setting clear stop-loss and take-profit levels can help manage risks effectively.

3. Conservative Strategy:
- Long-Term Focus: The conservative strategy aims at long-term growth and stability, minimizing short-term volatility but exposing the portfolio to long-term market risks.
- Mitigation Strategies: Applying dollar-cost averaging, reinvesting dividends, and using limit orders for buying and selling AAPL can mitigate risks associated with the conservative strategy.

Overall, continuous monitoring of market trends, news, and external factors is crucial for adapting the trading strategies and aligning them with the user's risk tolerance level. By implementing the suggested mitigation strategies, the proposed trading plans for AAPL can enhance returns and safeguard against potential losses.